In [4]:
from adam_impact_study.types import (
    ImpactorOrbits,
    Observations,
    ResultsTiming,
    WindowResult,
    ImpactorResultSummary,
)
from adam_impact_study.analysis.main import (
    run_all_analysis,
    collect_all_results,
    summarize_impact_study_results,
    make_analysis_plots,
)
from adam_impact_study.analysis.collect import collect_all_window_results_new, create_missing_window_results, collect_all_observations

ImportError: cannot import name 'collect_all_window_results_new' from 'adam_impact_study.analysis.main' (/Users/aleck/Code/adam_impacts_study/src/adam_impact_study/analysis/main.py)

In [2]:
run_dir = "../march_run/results"
out_dir = "../march_analysis/"

In [3]:
# run_all_analysis(run_dir, out_dir, summary_plots=True, individual_plots=False)

In [ ]:
impactor_orbits, observations, results_timing, window_results = collect_all_results(
    run_dir
)
impactor_orbits.to_parquet(
    "../march_analysis/impactor_orbits.parquet"
)
observations.to_parquet(
    "../march_analysis/observations.parquet"
)
results_timing.to_parquet(
    "../march_analysis/results_timing.parquet"
)
window_results.to_parquet(
    "../march_analysis/window_results.parquet"
)


In [5]:
impactor_orbits = ImpactorOrbits.from_parquet(
    "../march_analysis/impactor_orbits.parquet"
)
observations = Observations.from_parquet(
    "../march_analysis/observations.parquet"
)
results_timing = ResultsTiming.from_parquet(
    "../march_analysis/results_timing.parquet"
)
window_results = WindowResult.from_parquet(
    "../march_analysis/window_results.parquet"
)


In [ ]:
summary_results = summarize_impact_study_results(
    impactor_orbits, observations, results_timing, window_results, out_dir, max_processes=1
)
summary_results.to_parquet(
    "../march_analysis/summary_results.parquet"
)


In [7]:
summary_results = ImpactorResultSummary.from_parquet(
    "../march_analysis/summary_results.parquet"
)



In [ ]:
make_analysis_plots(summary_results, window_results, out_dir)